In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import torch
import pickle
import gensim
import pandas as pd
import numpy as np

from collections import OrderedDict
from torch import nn, optim
from torchtext.data import utils
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [24]:
from dataset import AbstractDataset
from util import save_model, load_model_and_opt, batch_predict
from cnn_util import CNNBase2, train_model

In [25]:
DEVICE = (f'cuda:0' if torch.cuda.is_available() else 'cpu')
HOME = '/home/hice1/khom9/CSE 8803 BMI Final Project'
EMBED_KEYS_PATH = f'{HOME}/wordvectors/abstracts200_normalized.wordvectors'

In [26]:
# Type of CNN (1, 2, or 3); refer to cnn_util.py for more info
VERSION = 2

In [27]:
print(f'Using device {DEVICE}')
if torch.cuda.is_available():
    print(torch.cuda.get_device_name())

Using device cuda:0
NVIDIA L40S


In [28]:
# Only run once

'''
from nltk.stem import WordNetLemmatizer
import nltk
from gensim.models import Word2Vec


df = pd.read_csv('CleanedAVdata.csv')
nltk.download('wordnet')
tk = utils.get_tokenizer('spacy')
lemma = WordNetLemmatizer()
abstracts = df['Abstract']
embed_dim = 200
tokens = pd.Series([[lemma.lemmatize(w) for w in tk(abst)] for abst in abstracts])
model = Word2Vec(sentences=tokens, vector_size=embed_dim, window=5, min_count=1, workers=12)

mu = np.mean(model.wv.vectors)
sigma = np.sqrt(np.var(model.wv.vectors))
model.wv.vectors = (1 + (np.clip(model.wv.vectors, mu-3*sigma, mu+3*sigma) - mu) / (3*sigma)) / 2
model.wv.vectors = np.clip(model.wv.vectors, 0, 1)

model.wv['\0'] = np.zeros(embed_dim)
model.wv.save(f'{HOME}/wordvectors/abstracts{embed_dim}_normalized.wordvectors')
'''



"\nfrom nltk.stem import WordNetLemmatizer\nimport nltk\nfrom gensim.models import Word2Vec\n\n\ndf = pd.read_csv('CleanedAVdata.csv')\nnltk.download('wordnet')\ntk = utils.get_tokenizer('spacy')\nlemma = WordNetLemmatizer()\nabstracts = df['Abstract']\nembed_dim = 200\ntokens = pd.Series([[lemma.lemmatize(w) for w in tk(abst)] for abst in abstracts])\nmodel = Word2Vec(sentences=tokens, vector_size=embed_dim, window=5, min_count=1, workers=12)\n\nmu = np.mean(model.wv.vectors)\nsigma = np.sqrt(np.var(model.wv.vectors))\nmodel.wv.vectors = (1 + (np.clip(model.wv.vectors, mu-3*sigma, mu+3*sigma) - mu) / (3*sigma)) / 2\nmodel.wv.vectors = np.clip(model.wv.vectors, 0, 1)\n\nmodel.wv['\x00'] = np.zeros(embed_dim)\nmodel.wv.save(f'{HOME}/wordvectors/abstracts{embed_dim}_normalized.wordvectors')\n"

In [29]:
tk = utils.get_tokenizer('spacy')
wv = gensim.models.KeyedVectors.load(EMBED_KEYS_PATH, mmap='r')
null_word = '\0'
d = AbstractDataset('CleanedAVdata.csv', 'Abstract', 'IPCR Classifications', tk, wv.key_to_index,
                    null_word=null_word, min_len=30, verbose=True)

/usr/local/lib/python3.10/dist-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(
100%|██████████| 23250/23250 [00:13<00:00, 1763.58it/s]


In [30]:
lr = 1e-4
batch_size = 48
model = CNNBase2(EMBED_KEYS_PATH, null_word=null_word).to(DEVICE)

save_path = f'{HOME}/models/cnn_model-{VERSION}.pth'
act_path = f'{HOME}/models/cnn_model-{VERSION}-max-activations.pkl'
wv_out_path = f'{HOME}/wordvectors/abstracts200_trained_normalized_{VERSION}.wordvectors'

num_pos = d.labels.sum(axis=0, keepdim=True).to_dense()
pos_weight = (d.labels.shape[0] - num_pos) / num_pos
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight.to(DEVICE))
optimizer = optim.Adam(model.parameters(), lr=lr)

# model, optimizer = load_model_and_opt(model, optimizer, f'{HOME}/models/cnn_model-4.pth')

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.70)

In [43]:
epochs = 100
train_model(model, optimizer, d, loss_fn, epochs=epochs, batch_size=batch_size, save_freq=None, 
            save_path=save_path, scheduler=scheduler, device=DEVICE)

Learning rate: 1.6806999999999997e-05
Scheduler: <torch.optim.lr_scheduler.StepLR object at 0x15542d73dbd0>
Training for 100 epochs, with batch size=48
Using device: cuda:0
Saving model every 25 epochs to /home/hice1/khom9/CSE 8803 BMI Final Project/models/cnn_model-999.pth

-----Epoch 1/100-----


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [32]:
# save_model(save_path, model, optimizer, epochs)
# print(f'Saved to {save_path}')

In [33]:
i = 8721 #18622
txt, label = d[i]
label = label.unsqueeze(0)
print(loss_fn(model(txt.to(DEVICE)).detach(), label.to(DEVICE)).item())
print(torch.cat([model(txt.to(DEVICE)).detach(), label.to(DEVICE)]).T)


0.02605363354086876
tensor([[-6.0868e+00,  0.0000e+00],
        [-2.9881e+02,  0.0000e+00],
        [-7.8192e+01,  0.0000e+00],
        [-3.0177e+02,  0.0000e+00],
        [-1.9628e+02,  0.0000e+00],
        [-2.7280e+02,  0.0000e+00],
        [-2.6021e+02,  0.0000e+00],
        [-6.9971e+00,  0.0000e+00],
        [-3.0986e+01,  0.0000e+00],
        [-1.9503e+02,  0.0000e+00],
        [-3.0685e+01,  0.0000e+00],
        [-6.6394e+01,  0.0000e+00],
        [-1.7377e+02,  0.0000e+00],
        [-7.1118e+01,  0.0000e+00],
        [-2.3271e+02,  0.0000e+00],
        [-3.3649e+02,  0.0000e+00],
        [-8.4957e+01,  0.0000e+00],
        [-2.7943e+02,  0.0000e+00],
        [-4.3802e+02,  0.0000e+00],
        [-1.4315e+02,  0.0000e+00],
        [-6.1936e+00,  0.0000e+00],
        [-5.3692e+02,  0.0000e+00],
        [-3.6010e+02,  0.0000e+00],
        [-4.5273e+02,  0.0000e+00],
        [-1.9372e+02,  0.0000e+00],
        [-5.3827e+02,  0.0000e+00],
        [-3.0033e+02,  0.0000e+00],
        

In [34]:
pred = (torch.sigmoid(batch_predict(model, d.abst_data, device=DEVICE).detach().cpu()) > 0.5).type(torch.float)
true = d.labels.to_dense()

In [35]:
total_loss = []
loss_fn_cpu = loss_fn.cpu()
for i in range(len(d)):
    total_loss.append(loss_fn_cpu(pred[i].unsqueeze(0), true[i].unsqueeze(0)).item())
    
print(f'Total avg loss: {np.mean(total_loss)}')

Total avg loss: 0.9919314197212137


In [36]:
x = pd.Series(total_loss)
x.sort_values(ascending=False).head(15)

11919    98.332405
130      89.000214
2267     87.721077
22573    84.880768
18414    84.525742
17995    84.115845
18534    82.558167
18548    81.296745
21263    80.901932
17761    80.785957
18622    80.739159
18642    80.721985
20998    44.534679
11239    44.242897
18483    41.305679
dtype: float64

In [37]:
print(precision_score(true, pred, average=None))
print(f'Total precision: {precision_score(true, pred, average="weighted")}')

[0.55252918 1.         0.75       1.         1.         1.
 0.43478261 0.63128492 0.59634551 0.8125     0.95       0.65625
 1.         0.76923077 0.83333333 1.         0.59770115 0.5
 1.         0.57971014 0.60499266 1.         0.66666667 1.
 0.58536585 1.         1.         0.86666667 0.5        0.5
 0.78249574 0.84482759 0.51039427 0.66666667 0.54946996 0.46056782
 0.57309942 1.         1.         1.         0.42857143 1.
 0.26315789 1.         0.75       0.9        1.         0.33333333
 1.         0.33333333 1.         0.33333333 1.         0.77192982
 0.76470588 0.75       0.67123288 0.76237624 0.85714286 0.66666667
 0.97142857 0.65263158 1.         0.68       0.7        0.51754386
 1.         0.96491228 0.78571429 0.65517241 1.         0.76923077
 0.78571429 0.75       0.54318154 0.70555556 0.875      0.95238095
 0.67994018 0.71757833 0.56105749 0.51044158 0.69086022 0.80088496
 0.84615385 0.79761905 0.72363636 0.52952381 0.77777778 0.62256809
 0.73643411]
Total precision: 0.6706

In [38]:
print(recall_score(true, pred, average=None))
print(f'Total recall: {recall_score(true, pred, average="weighted")}')

[1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 0.79508332 1.         0.99510832 1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         0.84024664 1.         1.         1.
 0.75992478 0.79017544 0.99479167 0.90631192 1.         1.
 1.         1.         1.         1.         1.         0.91874821
 1.        ]
Total recall: 0.8392411880872115


In [39]:
print(f1_score(true, pred, average='weighted'))

0.7371504904929346


In [40]:
print(f'Total accuracy: {accuracy_score(true, pred)}')

Total accuracy: 0.28658064516129034


In [41]:
wv_tuned = gensim.models.KeyedVectors.load(EMBED_KEYS_PATH, mmap='r')
wv_tuned.vectors = model.embedding.weight.data.detach().cpu().numpy()

wv_tuned.vectors = np.clip(wv_tuned.vectors, a_min=0, a_max=1.)

wv_tuned.save(wv_out_path)
print(f'Saved word embeddings to {wv_out_path}')

Saved word embeddings to /home/hice1/khom9/CSE 8803 BMI Final Project/wordvectors/abstracts200_trained_normalized_999.wordvectors


In [42]:
activations = []
model.eval()
with torch.no_grad():
    for txt, label in (d):
        outputs = list(model(txt.to(DEVICE), all_outputs_max=True))
        activations.append(outputs)
activations = torch.tensor(activations)

max_act = torch.max(activations, axis=0)[0]
max_act_dict = OrderedDict(zip(list(dict(model.named_modules()).keys())[1:], max_act))

output = open(act_path, 'wb')
pickle.dump(max_act_dict, output)
output.close()
print(f'Wrote max layer activations to {act_path}')

Wrote max layer activations to /home/hice1/khom9/CSE 8803 BMI Final Project/models/cnn_model-999-max-activations.pkl
